In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv("lending_data.csv")

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X = lending_data_df.drop(columns=['loan_status'])

In [4]:
# Review the y variable Series
y.sample(10)

54873    0
50495    0
22924    0
23559    0
53851    0
74613    0
42055    0
50318    0
31146    0
47011    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
model_LR = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
logistic_regression_pred=model_LR.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
logistic_acc_score = balanced_accuracy_score(y_test, logistic_regression_pred)
print(logistic_acc_score)

0.9442676901753825


In [11]:
# Generate a confusion matrix for the model
logistic_regression_conf_matrix= confusion_matrix(y_test, logistic_regression_pred)
print(logistic_regression_conf_matrix)

[[18679    80]
 [   67   558]]


In [12]:
# Print the classification report for the model
target_names=('healthy loan','high-risk loan')

print(classification_report(y_test, logistic_regression_pred,target_names=target_names))

                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** As we can see from the classification report, that the accuracy  is 99% and the balanced accuracy is 94%.
the percision for healthy loan is 100% and for high risk loan is 87%.
the percision for high risk loan is less than 90%, however, the model was able to give good prediction of the  loan risks.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [17]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
Random_oversampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_resampled, y_resampled = Random_oversampler_model.fit_resample(X_train, y_train)

In [18]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [19]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_model.fit(x_resampled, y_resampled)

# Make a prediction using the testing data
y_pred = classifier_model.predict(x_resampled)
classifier_model_df=pd.DataFrame({"Prediction": y_pred, "Actual": y_resampled})
classifier_model_df.sample(10)

,Prediction,Actual
42516,0,0
19565,0,0
66923,1,1
26036,0,0
84270,1,1
46821,0,0
109447,1,1
48371,0,0
105576,1,1
23757,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [21]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_resampled, y_pred))

0.994180571103648


In [22]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_resampled, y_pred))

[[55945   332]
 [  323 55954]]


In [23]:
# Print the classification report for the model
target_names=('healthy loan','high-risk loan')
print(classification_report(y_resampled, y_pred,target_names=target_names))

                precision    recall  f1-score   support

  healthy loan       0.99      0.99      0.99     56277
high-risk loan       0.99      0.99      0.99     56277

      accuracy                           0.99    112554
     macro avg       0.99      0.99      0.99    112554
  weighted avg       0.99      0.99      0.99    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** As we can see from the classification report, that the accuracy is 99% and the balanced accuracy is 99.4%. which is higher than the logistic regression.
 the percision for healthy loan is 99% and for high risk loan is 99%. however, the model was able to give good prediction of the loan risks.
 the prediction of high risk loan g=for this model ui better than the previous one.